# TP 1 - Manipulation des images

## Image et couleur

### 1. Affichage des différents canaux

L'objectif de ce TP est de vous faire manipuler les canaux couleur d'une image.

**Exercice**: Complétez ce code pour afficher les 3 canaux R, G, B de l'image.
Consultez la documentation de `plt.imshow` pour afficher ces canaux en niveaux de gris.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import skimage

#Chemin vers l'image
filename = "data/home.jpg"

#On ouvre l'image: la variable image renferme une matrice numpy.
image = skimage.io.imread(filename)

#Différents attributs intéressants
shape = image.shape
nb_dim = image.ndim

#Accès à l'intensité du premier pixel
intensity_first_pixel = image[0, 0]

print("La taille de l'image (hauteur, largeur, canaux) est ", shape)

#On affiche l'image
plt.imshow(image)
plt.axis("off")
plt.show()

### 2. Manipulation de l'espace YCbCr

L'espace YCbCr sépare la luminance (canal **Y**) et les chrominances (**Cb** et **Cr**)

Les formules pour l'obtention des canaux sont les suivantes:

Y = 0.299R + 0.587G + 0.114B
Cb = 0.564 * (B - Y) + 128
Cb = 0.713 * (R - Y) + 128

**Exercice**: Complétez le code suivant :
* Décomposez l'image précédente dans l'espace YCbCr et affichez ces 3 canaux.
* Comparez le canal **Y** avec la moyenne des canaux RGB. 

In [ ]:
#Y = ...
#Cb = ...
#Cr = ...

mean_rgb = np.mean(image, axis=2)

fig, ax = plt.subplots(1, 4)
ax[0].imshow(mean_rgb)

#On enlève les graduations pour chaque axe
[axi.set_axis_off() for axi in ax]
plt.show()

### 3. Effet esquisse

On souhaite obtenir un effet d'esquisse de notre image, de façon similaire à l'image suivante:

<table>
    <tr>
    <td>
        <img src="data/home.jpg" width="400" /> 
    </td>
    <td>    
        <img src="data/home_esquisse.png" width="400" /> </td>
    </tr>
</table>

L'effet esquisse est obtenu en suivant les étapes suivantes:
1. Extraction d'une image de contours $C$
2. Pondération du canal $Y$ par les contours $C$ :  $Y' = (1-\alpha) * C + \alpha * Y$ où $\alpha \in [0, 1]$.
3. Reconstruction d'une image RGB en utilisant le nouveau canal $Y'$.

**Exercice** : Complétez le code suivant.

**NB**: concernant l'étape 3, on pourra utiliser la fonction `ycbcr2rgb` du module `color` de `skimage`. 

Cette fonction attend une matrice HxLxC où C sont les canaux Y, Cb et Cr. On pourra la créer en utilisant la fonction `np.stack`.

In [ ]:
#On extrait l'image de contours par
#l'algorithme de Canny
C = skimage.feature.canny(Y/255)

#On inverse l'image de contours
C = (1-C)*255

alpha = 0.5

# Y_new = ...

#Conversion de l'image Y'CbCr vers RGB

#Affichage de l'image

### 4. Bonus : Compression YCbCr

Un autre intérêt de l'espace YCbCr est que l'on peut compresser l'image avec une perte minimale d'information. Le principe est de réduire les informations de chrominance et de laisser le canal de luminance inchangé. La perception de la qualité d'une image par un observateur est en effet davantage sensible à la luminance qu'à la chrominance.

L'idée est ici de compresser les canaux Cb et Cr en réduisant la taille et restaurant ensuite les images à leur taille originale.


**Exercice**: Complétez le code suivant.

**NB**: Pour changer la taille d'une image, on pourra s'aider de la taille `resize` du module `transform` de `skimage`.

Pour réduire la taille, on pourra essayer diverses valeurs de division de la taille (cf. variable `ratio`).

In [ ]:
#Facteur de division de la taille
ratio = 2

#Changement taille du canal Cr et Cb
#skimage.transform.resize(mon_image, [hauteur/ratio, largeur/ratio])

#Restauration de la taille des canaux

#Conversion ycbcr vers RGB

#Affichage de l'image et comparaison avec l'image originale

### 5. Bonus: chroma-keying

Le "chroma-keying" ou "incrustation en chrominance" est une technique de fusion d'images en couleurs. La méthode consiste à isoler puis remplacer les pixels "de fond" d'une image, de couleur caractéristique, par les pixels correspondants d'une seconde image. Un exemple type est le fond vert dans les films, qui permet d'insérer un arrière-plan du choix du réalisateur. Son usage remonte aux débuts du cinéma couleur (années 1940).

<table>
<tr>
    <td>
        <img src="data/metro.jpg" width="300" /> 
    </td>
    <td>    
        <img src="data/people.jpg" width="300" /> 
    </td>
    <td>    
        <img src="data/resultat_chroma_keying.png" width="300" /> 
    </td>
</tr>
</table>

L'idée est ici d'obtenir un masque binaire (0-255) à partir d'un seuillage sur un canal Y, Cb ou Cr de l'image avec le fond bleu.

<img src="data/seuil.png" width="300" />

* Lorsque la valeur du masque est égale à 0, on utilise les intensités de l'image en avant-plan
* Lorsque la valeur du masque est égale à 255, on conserve les intensités de l'image en arrière-plan.


**Exercice**: Complétez le code suivant.
1. On convertit chaque image dans l'espace YCbCr (fonction associée de `skimage`). 
Affichez et analysez chaque canal. 
Quel canal semble le plus indiqué pour obtenir le masque binaire?
2. Seuillez l'image pour obtenir le masque binaire. Vous pouvez vous aider de la fonction `np.where`. Affichez le masque.
3. Créer une nouvelle image où chaque canal Y'Cb'Cr' résulte de la fusion des deux images, en utilisant le masque binaire.
4. Convertissez cette nouvelle image dans l'espace RGB et affichez-la.

In [ ]:
image_front = skimage.io.imread("data/people.jpg")
image_back = skimage.io.imread("data/metro.jpg")

image_front_ycbcr = skimage.color.rgb2ycbcr(image_front)
image_back_ycbcr = skimage.color.rgb2ycbcr(image_back)

# Détermination du masque binaire (0, 255) en fonction des canaux YCbCr
# mask = ...

# Détermination de l'image avec chroma keying
# image_new = ...

# On reconvertit vers RGB
# image_new = skimage.color.ycbcr2rgb(image_new)

# Affichage de l'image